```
Decimal pixel code srrrrtccc

s slice 1-4
rrrr row number 1-2078
t is type
    0 image
    1 early dark
    2 late dark
ccc is column 1-512
```

In [1]:
import numpy as np

In [2]:
def pix(n): 
    return np.array(range(1,n+1))

In [3]:
def flip(a):
    return a[::-1]

In [4]:
slice_base = 100000000
row_base = 10000
image_type = 0
early_type = 1000
late_type = 2000

In [5]:
def row(n):
    return row_base*n+np.concatenate((
            1 * slice_base + early_type + pix(11),
            2 * slice_base + early_type + flip(pix(11)),
            3 * slice_base + early_type + pix(11),
            4 * slice_base + early_type + flip(pix(11)),
            1 * slice_base + pix(512),
            2 * slice_base + flip(pix(512)),
            3 * slice_base + pix(512),
            4 * slice_base + flip(pix(512)),
            1 * slice_base + late_type + pix(11),
            2 * slice_base + late_type + flip(pix(11)),
            3 * slice_base + late_type + pix(11),
            4 * slice_base + late_type + flip(pix(11)),
        ))

In [6]:
image=np.array(list(map(row,range(1,2079))))

In [7]:
from astropy.io import fits

In [8]:
fits.writeto('order_test.fits', image, None, clobber=True)

In [9]:
def slice_row(n):
    return row_base*n+np.concatenate((
            early_type + pix(11),
            pix(512),
            late_type + pix(11),
            ))


In [10]:
def slice_pixels(n):
    return n*slice_base+np.array(list(map(slice_row,range(1,2079))))

In [11]:
from httm.fits_utilities.raw_fits import raw_converter_from_fits

raw_data = raw_converter_from_fits('order_test.fits')

No handlers could be found for logger "httm.fits_utilities.header_settings"


In [12]:
for i in range(0,4):
    assert np.array_equal(slice_pixels(i+1),raw_data.slices[i].pixels), \
           "Slice from FITS does not match expected code"